**LSTM without new**

In [1]:

import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

In [ ]:
import os
print(os.getcwd())

/content


In [2]:
split = (0.85);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=1
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0

dataframe = pd.read_csv("/content/fpt.csv")
cols = ['Adj Close']

len_dataframe=dataframe.shape[0]

In [3]:
# Loại bỏ các hàng chứa NaN
dataframe = dataframe.dropna()

# Hoặc nếu bạn muốn loại bỏ các cột chứa NaN:
dataframe = dataframe.dropna(axis=1)

# Lưu lại file CSV sau khi loại bỏ NaN
dataframe.to_csv("/content/fpt_cleaned.csv", index=False)

# Kiểm tra lại dữ liệu sau khi loại bỏ NaN
print(dataframe.head())

   Unnamed: 0      Date  Adj Close  cafef_mean_compound  \
0         0.0  2/1/2024       96.9             0.212000   
1         1.0  3/1/2024       95.6             0.000000   
2         2.0  4/1/2024       96.3             0.074667   
3         3.0  5/1/2024       97.3             0.435667   
4         4.0  8/1/2024       97.4            -0.167433   

   vietstock_mean_compound  economy_mean_compound  ktck_mean_compound  
0                  0.12440               0.098667            0.148000  
1                 -0.13660               0.403300            0.064125  
2                  0.15910               0.230825            0.083733  
3                 -0.15910               0.217200           -0.120400  
4                  0.62885               0.415650            0.058475  


In [4]:
print(dataframe.columns)


Index(['Unnamed: 0', 'Date', 'Adj Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [5]:
i_split = int(len(dataframe) * split)
data_train = dataframe.get(cols).values[:i_split]
data_test  = dataframe.get(cols).values[i_split:]
len_train  = len(data_train)
len_test   = len(data_test)
len_train_windows = None
print('data_train.shape',data_train.shape)
print('data_test.shape',data_test.shape)

data_train.shape (120, 1)
data_test.shape (22, 1)


In [6]:
#get_test_data
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
 # get original y_test
y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

y_test_ori.shape (12, 1)


In [7]:
#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)


In [8]:
# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)


x_test.shape (12, 9, 1)
y_test.shape (12, 1)


In [9]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)


In [10]:
# normalised_data=window_data
data_windows=normalised_data
x_train = data_windows[:, :-1]
y_train = data_windows[:, -1,[0]]
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)


x_train.shape (110, 9, 1)
y_train.shape (110, 1)


In [ ]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())


0 0
0 0


In [ ]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [11]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.5400
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.4768 
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.3906
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.3173
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.2499


In [ ]:
import numpy as np
print(np.isnan(x_test).sum(), np.isinf(x_test).sum())
print(np.isnan(y_test).sum(), np.isinf(y_test).sum())

0 0
0 0


In [ ]:
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

In [12]:
model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 630ms/step


array([[0.4871814 ],
       [0.5019691 ],
       [0.5069985 ],
       [0.56911486],
       [0.60734516],
       [0.60134923],
       [0.6554734 ],
       [0.6847862 ],
       [0.65182006],
       [0.6152994 ],
       [0.553807  ],
       [0.4848247 ]], dtype=float32)

In [13]:
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

print(prediction_seqs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[array([0.48718148], dtype=float32)], [array([0.5019691], dtype=float32)], [array([0.5069986], dtype=float32)], [array([0.5691148], dtype=float32)], [array([0.6073451], dtype=float32)], [array([0.60134923], dtype=float32)], [array([0.6554735], dtype=float32)], [array([0.68478626], dtype=float32)], [array([0.65182006], dtype=float32)], [array([0.6152992], dtype=float32)], [array([0.5538071], dtype=float32)], [array([0.48482472], dtype=float32)]]


In [14]:
#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)



[133.7154055893421, 134.81693425178528, 135.64899019002914, 135.7821839928627, 134.97712359428405, 133.877674973011, 132.17575552463532, 131.2000335931778, 130.58783662319183, 130.22993240356445, 129.62730938196182, 128.95128222703934]


In [15]:
error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)

MSE [22.60804554]
accuracy [0.96989769]
mean_error_percent [0.03010231]


**LSTM with new**

In [21]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM, Input
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats


split = (0.85);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=5
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0

dataframe = pd.read_csv("/content/fpt_cleaned.csv")
print(dataframe.columns)

Index(['Unnamed: 0', 'Date', 'Adj Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [22]:
cols = ['Adj Close','cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']

len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_train = dataframe.get(cols).values[:i_split]
data_test  = dataframe.get(cols).values[i_split:]
len_train  = len(data_train)
len_test   = len(data_test)
len_train_windows = None
print('data_train.shape',data_train.shape)
print('data_test.shape',data_test.shape)


data_train.shape (120, 5)
data_test.shape (22, 5)


In [23]:

data_train[0:5]

array([[ 9.69000000e+01,  2.12000000e-01,  1.24400000e-01,
         9.86666670e-02,  1.48000000e-01],
       [ 9.56000000e+01,  0.00000000e+00, -1.36600000e-01,
         4.03300000e-01,  6.41250000e-02],
       [ 9.63000000e+01,  7.46666670e-02,  1.59100000e-01,
         2.30825000e-01,  8.37333330e-02],
       [ 9.73000000e+01,  4.35666667e-01, -1.59100000e-01,
         2.17200000e-01, -1.20400000e-01],
       [ 9.74000000e+01, -1.67433333e-01,  6.28850000e-01,
         4.15650000e-01,  5.84750000e-02]])

In [24]:
data_test[0:5]

array([[ 1.28600000e+02,  1.48000000e-01,  2.91866667e-01,
         2.86000000e-02,  8.44200000e-01],
       [ 1.30500000e+02,  5.26700000e-01,  1.06066667e-01,
        -8.50333330e-02,  2.36700000e-01],
       [ 1.32100000e+02,  0.00000000e+00, -2.55500000e-02,
         1.61600000e-01,  2.12000000e-01],
       [ 1.36100000e+02,  0.00000000e+00,  2.25933333e-01,
         3.74333333e-01, -8.60000000e-03],
       [ 1.38500000e+02,  0.00000000e+00, -1.46666670e-02,
         1.80800000e-01,  2.32633333e-01]])

In [25]:
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
 # get original y_test
y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

y_test_ori.shape (12, 1)


In [26]:
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

In [27]:
#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)

x_test.shape (12, 9, 5)
y_test.shape (12, 1)


In [28]:
x_test[0]

array([[ 0.        ,  0.148     ,  0.29186667,  0.0286    ,  0.8442    ],
       [ 0.18095238,  0.5267    ,  0.10606667, -0.08503333,  0.2367    ],
       [ 0.33333333,  0.        , -0.02555   ,  0.1616    ,  0.212     ],
       [ 0.71428571,  0.        ,  0.22593333,  0.37433333, -0.0086    ],
       [ 0.94285714,  0.        , -0.01466667,  0.1808    ,  0.23263333],
       [ 1.        ,  0.4144    ,  0.4378    ,  0.26013333, -0.21823333],
       [ 0.7047619 ,  0.2294    , -0.0334    ,  0.0911    ,  0.148     ],
       [ 0.52380952,  0.2997    , -0.17      ,  0.17963333,  0.2027    ],
       [ 0.34285714,  0.25825   , -0.1591    , -0.2276    , -0.0593    ]])

In [29]:
y_test

array([[0.46666667],
       [0.36046512],
       [0.27142857],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.06521739],
       [0.        ],
       [0.17346939],
       [0.59183673],
       [0.48979592]])

In [30]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train = data_windows[:, :-1]
y_train = data_windows[:, -1,[0]]
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)

x_train.shape (110, 9, 5)
y_train.shape (110, 1)


In [31]:
x_train[0]

array([[ 0.81481481,  0.212     ,  0.1244    ,  0.09866667,  0.148     ],
       [ 0.33333333,  0.        , -0.1366    ,  0.4033    ,  0.064125  ],
       [ 0.59259259,  0.07466667,  0.1591    ,  0.230825  ,  0.08373333],
       [ 0.96296296,  0.43566667, -0.1591    ,  0.2172    , -0.1204    ],
       [ 1.        , -0.16743333,  0.62885   ,  0.41565   ,  0.058475  ],
       [ 0.96296296,  0.29086667,  0.        ,  0.2884    ,  0.        ],
       [ 0.74074074,  0.53505   ,  0.        ,  0.35775   ,  0.06906667],
       [ 0.        ,  0.3803    ,  0.2787    ,  0.2971    , -0.33525   ],
       [ 0.03703704,  0.26026667, -0.06005   ,  0.318     ,  0.2732    ]])

In [32]:
y_train[0:5]

array([[0.11111111],
       [0.        ],
       [0.33333333],
       [0.37037037],
       [0.48148148]])

In [33]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

0 0
0 0


In [34]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)


In [35]:
# LSTM MODEL
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input
model = Sequential()
model.add(Input(shape=(input_timesteps, input_dim)))
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)



Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.5221
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.4471
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3652
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2811
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2070


In [37]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)


model.predict(x_test)


#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

print(prediction_seqs)


#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)


# In[33]:


error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)



Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 0.4962
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.4205 
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.3224 
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2375 
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1629 
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[array([0.73233074], dtype=float32)], [array([0.70120394], dtype=float32)], [array([0.6704203], dtype=float32)], [array([0.7373308], dtype=float32)], [array([0.77595705], dtype=float32

**DP-LSTM**

In [39]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats


df = pd.read_csv("/content/fpt_cleaned.csv")

cafef_var=np.var(df.cafef_mean_compound)
vietstock_var=np.var(df.vietstock_mean_compound)
economy_var=np.var(df.economy_mean_compound)
ktck_var=np.var(df.ktck_mean_compound)

mu=0

noise=0.1

sigma_cafef=noise*cafef_var
sigma_vietstock=noise*vietstock_var
sigma_economy=noise*economy_var
sigma_ktck=noise*ktck_var

n=df.shape[0]

df_noise=pd.DataFrame()

df_noise['cafef_noise']=df['cafef_mean_compound']
df_noise['vietstock_noise']=df['vietstock_mean_compound']
df_noise['economy_noise']=df['economy_mean_compound']
df_noise['ktck_noise']=df['ktck_mean_compound']

for i in range(0,n):
    df_noise['cafef_noise'][i]+=np.random.normal(mu,sigma_cafef)
    df_noise['vietstock_noise'][i]+=np.random.normal(mu,sigma_vietstock)
    df_noise['economy_noise'][i]+=np.random.normal(mu,sigma_economy)
    df_noise['ktck_noise'][i]+=np.random.normal(mu,sigma_ktck)


df_noise.to_csv("/content/source_price_noise0.csv")
dfn=pd.read_csv("/content/source_price_noise0.csv",index_col=0)

In [40]:
df_1n=pd.DataFrame()
df_1n['cafef']=df_noise['cafef_noise']
df_1n['vietstock']=df['vietstock_mean_compound']
df_1n['economy']=df['economy_mean_compound']
df_1n['ktck']=df['ktck_mean_compound']
df_1n['price']=df['Adj Close']

df_2n=pd.DataFrame()
df_2n['cafef']=df['cafef_mean_compound']
df_2n['vietstock']=df_noise['vietstock_noise']
df_2n['economy']=df['economy_mean_compound']
df_2n['ktck']=df['ktck_mean_compound']
df_2n['price']=df['Adj Close']

df_3n=pd.DataFrame()
df_3n['cafef']=df['cafef_mean_compound']
df_3n['vietstock']=df['vietstock_mean_compound']
df_3n['economy']=df_noise['economy_noise']
df_3n['ktck']=df['ktck_mean_compound']
df_3n['price']=df['Adj Close']

df_4n=pd.DataFrame()
df_4n['cafef']=df['cafef_mean_compound']
df_4n['vietstock']=df['vietstock_mean_compound']
df_4n['economy']=df['economy_mean_compound']
df_4n['ktck']=df_noise['ktck_noise']
df_4n['price']=df['Adj Close']

df1=df_1n
df2=df_2n
df3=df_3n
df4=df_4n

In [41]:

split = (0.85)
sequence_length=10;
normalise= True
batch_size=100;
input_dim=5
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0


# In[5]:


i_split = int(len(df1) * split)

cols = ['price','cafef','vietstock','economy','ktck']
data_train_1 = df1.get(cols).values[:i_split]
data_train_2 = df2.get(cols).values[:i_split]
data_train_3 = df3.get(cols).values[:i_split]
data_train_4 = df4.get(cols).values[:i_split]

len_train  = len(data_train_1)
len_train_windows = None


# In[6]:


#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_1[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train1 = data_windows[:, :-1]
y_train1 = data_windows[:, -1,[0]]
print('x_train1.shape',x_train1.shape)
print('y_train1.shape',y_train1.shape)


x_train1.shape (110, 9, 5)
y_train1.shape (110, 1)


In [42]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_2[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train2 = data_windows[:, :-1]
y_train2 = data_windows[:, -1,[0]]
print('x_train2.shape',x_train2.shape)
print('y_train2.shape',y_train2.shape)


x_train2.shape (110, 9, 5)
y_train2.shape (110, 1)


In [43]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_3[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train3 = data_windows[:, :-1]
y_train3 = data_windows[:, -1,[0]]
print('x_train3.shape',x_train3.shape)
print('y_train3.shape',y_train3.shape)

x_train3.shape (110, 9, 5)
y_train3.shape (110, 1)


In [44]:

#get_train_data

data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_4[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train4 = data_windows[:, :-1]
y_train4 = data_windows[:, -1,[0]]
print('x_train4.shape',x_train4.shape)
print('y_train4.shape',y_train4.shape)

x_train4.shape (110, 9, 5)
y_train4.shape (110, 1)


In [45]:

##concat train window
type(x_train4)


numpy.ndarray

In [46]:

x_train_t=np.concatenate((x_train1,x_train2,x_train3,x_train4),axis=0)
print(x_train_t.shape)
x_train=x_train_t

y_train_t=np.concatenate((y_train1,y_train2,y_train3,y_train4),axis=0)
print(y_train_t.shape)
y_train=y_train_t


(440, 9, 5)
(440, 1)


In [48]:
dataframe = pd.read_csv("/content/fpt_cleaned.csv")
print(dataframe.columns)  # Check the actual number of columns

Index(['Unnamed: 0', 'Date', 'Adj Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [50]:
dataframe= pd.read_csv("/content/fpt_cleaned.csv")
dataframe.columns=['Unnamed: 0', 'Date', 'Adj Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']
cols = ['Adj Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']
len_dataframe=dataframe.shape[0]

In [51]:

len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_test  = dataframe.get(cols).values[i_split:]

len_test   = len(data_test)
len_train_windows = None

print('data_test.shape',data_test.shape)

data_test.shape (22, 5)


In [52]:
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])

data_windows = np.array(data_windows).astype(float)

y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)


y_test_ori.shape (12, 1)


In [53]:
y_test_ori

array([[133.5],
       [133.6],
       [134. ],
       [131.4],
       [128.6],
       [126. ],
       [124.9],
       [125.5],
       [124.2],
       [125.9],
       [130. ],
       [129. ]])

In [54]:
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)


x_test.shape (12, 9, 5)
y_test.shape (12, 1)


In [55]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())

0 0
0 0


In [56]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [57]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - loss: 0.5324
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.3526
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.1762
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1955
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1439


In [58]:

#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [59]:

#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

[134.6589602291584, 135.14298011064528, 135.6843585729599, 135.54349670410156, 134.57117805480956, 133.61987135410308, 131.5927864253521, 130.53797113895416, 129.98155113458634, 129.98920259475707, 129.76376519203185, 129.28875373601915]


In [60]:
 np.save('/content/sp_5dim_n01_7032.npy',de_predicted)

In [61]:
error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)



MSE [19.82572103]
accuracy [0.97099539]
mean_error_percent [0.02900461]
